In [3]:
import csv
import string
import time

community_words = ["thank", "thanks"]
outreach_words = ["review", "shar", "featured", "shout out", "promot", "honor"]
sell_words = ["hours left", "days left", "weeks left", "minutes left", "hours to go", "days to go", "weeks to go"
              "pledge", "reach our goals", "left to pledge", "away from", "early bird", "pre order", 'rewards',
             'funded', ]
sell_words_2 = ["fund", "back"]

detection = []
translator = str.maketrans('', '', string.punctuation)
remove = string.punctuation
remove = remove.replace("$", "")
remove = remove.replace("%", "")# don't remove hyphens

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString) and \
("$" in inputString or inputString.endswith("k") or "%" in inputString) and ("http" not in inputString)


class Campaign():
    def __init__(self, title, cid, post, url, source, postID):
        initial_dict = {'source': source, 'text': post, 'postID': postID}
        self._title = title
        self._cid = cid
        self._post = post
        self._url = url
        self._source = source
        self._postdict = {}
        self._listofdict = []
        self._listofdict.append(initial_dict)

    def addListOfDict(self,listofdict):
        self._listofdict = self._listofdict + listofdict

    def addListOfDict(self,listofdict):
        self._listofdict = listofdict

    def addPost(self,post,source, postID):
        tempdict = {}
        tempdict['text'] = post
        tempdict['source'] = source
        tempdict['postID'] = postID
        self._listofdict.append(tempdict)
        return self._listofdict

def openSmallDataFile():
    listOfCampaigns = []
    answers = []
    posts = []

            ##CHANGE DATA FILE TO RELEVENT NAME##
    with open('data_imput.csv', encoding = 'ascii', errors = 'ignore') as csvfile:
        reader = csv.reader(csvfile)
        contents = [row for row in reader]
        csvfile.close()

    prev_obj = None

    for cell in contents[1:]:

        cid = cell[1]
        title = cell[2]
        post = cell[6]
        url = cell[4]
        source = cell[5]
        postID = cell[3]
        answers.append(cell[7])

        # cid = cell[14]
        # title = cell[15]
        # post = cell[8]
        # url = cell[6]
        # source = cell[7]
        # postID = cell[1]

        if prev_obj == None:
            obj = Campaign(title, cid, post, url, source, postID)
            posts.append(postID)
            prev_obj = obj
        elif prev_obj._cid == cid:
            prev_obj.addPost(post, source, postID)
            posts.append(postID)
        else:
            listOfCampaigns.append(prev_obj)
            obj = Campaign(title, cid, post, url, source, postID)
            posts.append(postID)
            prev_obj = obj
    listOfCampaigns.append(prev_obj)
    return [listOfCampaigns, answers, posts]


def sorter(post_text):
    global special_words
    global sell_words
    global pattern
    post_text = str.lower(post_text)
    post_text = ' '.join(word.strip(remove) for word in post_text.split())

    #print(post_text)

    for word in post_text.split():
        if hasNumbers(word):
            detection.append("4")
            #print(word)
            #print("4")
            return "4"

    for word in sell_words:
        if word in post_text:
            detection.append("4")
            #print(word)
            #print("4")
            return "4"

    for word in outreach_words:
        if word in post_text:
            detection.append("3")
            #print(word)
            #print("3")
            return "3"

    for word in community_words:
        if word in post_text:
            detection.append("2")
            #print(word)
            #print("2")
            return "2"

    #print("1")
    detection.append("1")
    return "1"


def assign(posts_list):
    for post in posts_list:
        num = sorter(post['text'])
        post['sell'] = num
    return posts_list


def success_rate(answer_list, complete_post_list):
    f = open('output2.csv', 'w')
    writer = csv.writer(f)
    writer.writerow( ('CategoryNames','CategoryNumbers') )
    #print(len(detection))
    #print(len(complete_post_list))
    for i,j in zip(detection, complete_post_list):
        f.write(str(i)+","+str(j)+"\n")
    f.close()

    #print(detection)

    wrong = []
    right = 0
    wrongAnswers = []
    for det, ans, post in zip(detection, answer_list, complete_post_list):
        if det == ans:
            right += 1
        else:
            wrong.append(post)
            wrongAnswers.append(det)
    return [right/len(detection), wrong, wrongAnswers]


def main():
    output = openSmallDataFile() #Joe's code

    campaign_list = output[0]
    answer_list = output[1]
    complete_post_list = output[2]
    new_campaign_list = []

    start_time = time.time()

    for campaign in campaign_list:
        postsList = campaign._listofdict
        newPostsList = assign(postsList)
        campaign.posts = newPostsList
        new_campaign_list.append(campaign)

    output2 = success_rate(answer_list, complete_post_list)
    #outputcsv(output2[1], output2[2])
    #print(output2[0])

    elapsed_time = time.time() - start_time
    print("Elapsed time: " + str(elapsed_time))

if __name__ == '__main__':
    main()

Elapsed time: 0.046473026275634766
